In [65]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as lines
import matplotlib.colors as mcolors
from scipy.interpolate import CubicSpline

In [99]:
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']

In [118]:
# % VELOCITY PROFILE GENERATION
x_k_dot_final = 1                                                      # Final ball-bot velocity (m/s)
t_final = 20                                                           # Duration of the simulation (s)
t_constant = 6 # Ramp lasts from 2.5 s to t_constant                   # Time to reach constant final velocity (s)
dt = 1/200                                                             # Simulation timestep
t_length = int(t_final/dt)                                             # Length of time vector
t = np.linspace(0, t_final, t_length)                                  # Time vector (s)


linear = np.linspace(0,x_k_dot_final, (round(t_length*(t_constant/t_final))-500) ) 
x_k_dot = np.concatenate((np.zeros(500), linear, x_k_dot_final*np.ones(4000-1200)))                      # Velocity vector (m/s)
x_k_dot[np.arange(len(x_k_dot),t_length,1)] = x_k_dot_final          # Velocity vector (m/s)

plt.figure(dpi=300)
plt.plot(t,x_k_dot,colors[0])
plt.xlabel('Time(s)')
plt.ylabel('Velocity (m/s)')
smoothed_velocty = CubicSpline(t,x_k_dot)
type(smoothed_velocty(t))
plt.plot(t,smoothed_velocty(t),colors[1])
plt.axis([0,2.75,0,.2])
if x_k_dot == smoothed_velocty:

IndentationError: expected an indented block (Temp/ipykernel_5260/2641320148.py, line 22)